In [297]:
import pandas as pd
import numpy as np
import datetime
import math
import warnings
warnings.filterwarnings("ignore")

In [298]:
#read all data files
Lesson1 = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/lesson2017-01-01to2017-02-01.csv')
Lesson2 = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/lesson2017-02-01to2017-03-01.csv')
Lesson3 = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/lesson2017-02-25 to2017-04-10.csv')
Lesson4 = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/lesson2017-04-01 to2017-05-01.csv')
Lesson5 = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/lesson2017-05-01 to2017-06-01.csv')
Lesson6 = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/lesson2017-06-01 to2017-07-01.csv')

KissUserList = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissUserList.csv')
Receipt_data = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/receipt_data.csv') 
appversion_data = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/appversion_data.csv')

In [403]:
#Manuscripttype = pd.read_csv('/Users/stacyli/Project/Retention/Core_value_study/KissRawData/Manuscripttype.csv',encoding="latin-1")

In [ ]:
#We want to merge receipt data with Kissmetrics data to generate a table of CTP, M1-M6 retention result

In [299]:
Receipt_data.head()

,Unnamed: 0,iapExpiresDate,iapOriginalPurchaseDate,iapProductId,iapPurchaseDate,iapWebOrderLineItemId,parseUser,reportingStatus
0,0,2017-02-01 00:00:56,2016-12-31 18:01:08,sub_month,2017-01-01 00:00:56,210000055873133,B5cGAhsgDY,PAID_11
1,1,2017-02-01 00:05:26,2017-01-01 00:05:27,sub_month,2017-01-01 00:05:26,110000043750083,WD5qFPwRSk,TRIAL
2,2,2017-02-01 00:25:34,2017-01-01 00:25:34,sub_year,2017-01-01 00:25:34,700000025993902,pBzhf6dCpq,TRIAL
3,3,2017-02-01 00:36:22,2016-12-31 18:36:28,sub_month,2017-01-01 00:36:22,560000024653732,ooXqpYVS97,PAID_12
4,4,2017-02-01 01:11:02,2016-12-31 19:11:10,sub_month,2017-01-01 01:11:02,270000046825864,66tO2Lxm8F,PAID_26


In [300]:
KissUserList.head(2)
print(len(KissUserList))

18094


In [301]:
subscription_retention = KissUserList.merge(Receipt_data,how = 'left',left_on = 'Last value of Userid - ever', right_on = 'parseUser')

In [302]:
#remove 105 records
subscription_retention = subscription_retention[~subscription_retention['reportingStatus'].isnull()]

In [303]:
print(len(subscription_retention))

56151


We need to code CTP, PAID_1, PAID_2.... to PAID_26 timestamp

In [304]:
subscription_retention.loc[subscription_retention['reportingStatus'] == 'TRIAL','TRIAL_timestamp'] = subscription_retention.loc[subscription_retention['reportingStatus'] == 'TRIAL','iapPurchaseDate']

In [305]:
for i in range(1, 8):
    label = 'PAID_' + str(i)
    subscription_retention.loc[subscription_retention['reportingStatus'] == label,label + '_timestamp'] = subscription_retention.loc[subscription_retention['reportingStatus'] == label,'iapPurchaseDate']    

In [306]:
empty = []
for i in range(1, 8):
    label = 'PAID_' + str(i) + '_timestamp'
    empty.append(label)
empty.append('Identity')
empty.append('TRIAL_timestamp')

In [307]:
subscription_retention.head()

,ID,Identity,Last value of Userid - ever,Last value of Email - ever,First time of IAP Trial Purchase Success - ever,First value of Age - ever,Last value of Number childen - ever,Last value of Number children - ever,Last value of Gender - ever,Last value of Age - ever,...,parseUser,reportingStatus,TRIAL_timestamp,PAID_1_timestamp,PAID_2_timestamp,PAID_3_timestamp,PAID_4_timestamp,PAID_5_timestamp,PAID_6_timestamp,PAID_7_timestamp
0,2574321,acupofkizoku@gmail.com,soqFSEjaIv,acupofkizoku@gmail.com,"Jan 27, 2017 05:32:52",5,1.0,NaN,F,5,...,soqFSEjaIv,TRIAL,2017-01-27 05:32:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3198127,anthonyibanez559@icloud.com,tzGwL6Q6Lm,anthonyibanez559@icloud.com,"Mar 05, 2017 22:48:12",8,1.0,NaN,M,8,...,tzGwL6Q6Lm,TRIAL,2017-03-05 22:48:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3240572,caitlin.perlman@gmail.com,1xD4BR1Ryo,caitlin.perlman@gmail.com,"Mar 16, 2017 20:46:43",4,1.0,NaN,NaN,4,...,1xD4BR1Ryo,TRIAL,2017-03-16 20:46:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3246206,carlapondler1@gmail.com,LieQaYZsLl,carlapondler1@gmail.com,"Mar 18, 2017 10:08:52",NaN,1.0,NaN,NaN,NaN,...,LieQaYZsLl,PAID_1,NaN,2017-03-10 17:23:42,NaN,NaN,NaN,NaN,NaN,NaN
4,3246206,carlapondler1@gmail.com,LieQaYZsLl,carlapondler1@gmail.com,"Mar 18, 2017 10:08:52",NaN,1.0,NaN,NaN,NaN,...,LieQaYZsLl,PAID_2,NaN,NaN,2017-04-10 17:23:42,NaN,NaN,NaN,NaN,NaN


In [308]:
subscription_timestamp = subscription_retention[empty].groupby(['Identity']).agg(lambda x: min(x[~x.isnull()]) if len(x[~x.isnull()])>0 else None)

In [309]:
subscription_timestamp.head()

,PAID_1_timestamp,PAID_2_timestamp,PAID_3_timestamp,PAID_4_timestamp,PAID_5_timestamp,PAID_6_timestamp,PAID_7_timestamp,TRIAL_timestamp
Identity,,,,,,,,
acupofkizoku@gmail.com,None,None,None,None,None,None,None,2017-01-27 05:32:21
anthonyibanez559@icloud.com,None,None,None,None,None,None,None,2017-03-05 22:48:06
caitlin.perlman@gmail.com,None,None,None,None,None,None,None,2017-03-16 20:46:41
carlapondler1@gmail.com,2017-03-10 17:23:42,2017-04-10 17:23:42,2017-05-10 17:23:42,2017-06-10 17:23:42,2017-07-10 17:23:42,None,None,None
caroline.lee27@gmail.com,2017-02-18 08:34:11,2017-03-18 08:34:11,None,None,None,None,None,2017-01-18 08:34:11


In [54]:
#sub_result = subscription_retention.groupby(['Identity'])['reportingStatus'].value_counts().unstack(fill_value = 0)

In [310]:
#sub_result.head()

In [311]:
KissUserList = KissUserList.merge(subscription_timestamp, how = 'inner', left_on = 'Identity', right_index = True)

In [313]:
KissUserList.head()

,ID,Identity,Last value of Userid - ever,Last value of Email - ever,First time of IAP Trial Purchase Success - ever,First value of Age - ever,Last value of Number childen - ever,Last value of Number children - ever,Last value of Gender - ever,Last value of Age - ever,First time of Pathways manuscript open - ever,First time of iOS Screen Load - ever,PAID_1_timestamp,PAID_2_timestamp,PAID_3_timestamp,PAID_4_timestamp,PAID_5_timestamp,PAID_6_timestamp,PAID_7_timestamp,TRIAL_timestamp
0,2574321,acupofkizoku@gmail.com,soqFSEjaIv,acupofkizoku@gmail.com,"Jan 27, 2017 05:32:52",5,1.0,NaN,F,5,NaN,NaN,None,None,None,None,None,None,None,2017-01-27 05:32:21
1,3198127,anthonyibanez559@icloud.com,tzGwL6Q6Lm,anthonyibanez559@icloud.com,"Mar 05, 2017 22:48:12",8,1.0,NaN,M,8,NaN,NaN,None,None,None,None,None,None,None,2017-03-05 22:48:06
2,3240572,caitlin.perlman@gmail.com,1xD4BR1Ryo,caitlin.perlman@gmail.com,"Mar 16, 2017 20:46:43",4,1.0,NaN,NaN,4,NaN,NaN,None,None,None,None,None,None,None,2017-03-16 20:46:41
3,3246206,carlapondler1@gmail.com,LieQaYZsLl,carlapondler1@gmail.com,"Mar 18, 2017 10:08:52",NaN,1.0,NaN,NaN,NaN,NaN,NaN,2017-03-10 17:23:42,2017-04-10 17:23:42,2017-05-10 17:23:42,2017-06-10 17:23:42,2017-07-10 17:23:42,None,None,None
4,2520676,caroline.lee27@gmail.com,WglP67MDxz,caroline.lee27@gmail.com,"Jan 18, 2017 08:34:13",6,1.0,NaN,NaN,6,NaN,NaN,2017-02-18 08:34:11,2017-03-18 08:34:11,None,None,None,None,None,2017-01-18 08:34:11


In [314]:
Lesson4.drop(['_c','_u'],axis = 1, inplace = True)
Lesson5.drop(['_c','_u'],axis = 1, inplace = True)
Lesson6.drop(['_c','_u'],axis = 1, inplace = True)

In [315]:
#Combine all three files and delete duplicates
Lesson  = Lesson1.append(Lesson2).append(Lesson3).append(Lesson4).append(Lesson5).append(Lesson6)
Lesson.drop('Unnamed: 0',axis = 1, inplace = True)
Lesson.drop_duplicates(inplace = True)

In [316]:
Lesson.head()
print(len(Lesson))

12785208


In [409]:
#Lesson = Lesson.merge(Manuscripttype, how = 'left', left_on = 'manuscripttitle',right_on = 'Manuscript Title')

In [317]:
KissUserList.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17991 entries, 0 to 18093
Data columns (total 20 columns):
ID                                                 17991 non-null int64
Identity                                           17991 non-null object
Last value of Userid - ever                        17989 non-null object
Last value of Email - ever                         17961 non-null object
First time of IAP Trial Purchase Success - ever    17991 non-null object
First value of Age - ever                          16395 non-null object
Last value of Number childen - ever                17427 non-null float64
Last value of Number children - ever               83 non-null float64
Last value of Gender - ever                        6266 non-null object
Last value of Age - ever                           16393 non-null object
First time of Pathways manuscript open - ever      703 non-null object
First time of iOS Screen Load - ever               699 non-null object
PAID_1_timestamp      

In [318]:
#Make column names more readable and drop unnecessary columns

Lesson.rename(columns = {'_n':'eventname',
                         '_p':'identity',
                         '_t':'timestamp'}, inplace = True)

Lesson.drop(['category','eventvalue','manuscriptpage','manuscriptunit','saturdaysurprisestory'], axis = 1, inplace = True)

#Convert epoch time to timestamp
Lesson['timestamp'] = [datetime.datetime.fromtimestamp(timestamp) for timestamp in Lesson['timestamp']]


#KissUserList
#Convert str to timestamp
KissUserList['First time of IAP Trial Purchase Success - ever'] = pd.to_datetime(KissUserList['First time of IAP Trial Purchase Success - ever'])
KissUserList.rename(columns = {'First time of IAP Trial Purchase Success - ever': 'IAP Trial Success',
                               'Last value of Userid - ever':'Userid',
                               'Last value of Email - ever':'Email',
                               'First value of Age - ever':'First Age',
                               'Last value of Number childen - ever':'Number Childen',
                               'Last value of Number children - ever': 'Number Children',
                               'Last value of Gender - ever': 'Gender',
                               'Last value of Age - ever': 'LastAge',
                               'First time of Pathways manuscript open - ever': 'Pathways manuscript open',
                               'First time of iOS Screen Load - ever':'Welcome_page'}, inplace = True)

In [319]:
KissUserList.head(2)

,ID,Identity,Userid,Email,IAP Trial Success,First Age,Number Childen,Number Children,Gender,LastAge,Pathways manuscript open,Welcome_page,PAID_1_timestamp,PAID_2_timestamp,PAID_3_timestamp,PAID_4_timestamp,PAID_5_timestamp,PAID_6_timestamp,PAID_7_timestamp,TRIAL_timestamp
0,2574321,acupofkizoku@gmail.com,soqFSEjaIv,acupofkizoku@gmail.com,2017-01-27 05:32:52,5,1.0,NaN,F,5,NaN,NaN,None,None,None,None,None,None,None,2017-01-27 05:32:21
1,3198127,anthonyibanez559@icloud.com,tzGwL6Q6Lm,anthonyibanez559@icloud.com,2017-03-05 22:48:12,8,1.0,NaN,M,8,NaN,NaN,None,None,None,None,None,None,None,2017-03-05 22:48:06


In [320]:
print(Lesson['timestamp'].max())
print(Lesson['timestamp'].min())
print(len(Lesson))

2017-06-30 20:00:00
2016-12-31 19:00:25
12785208


In [321]:
#merge Lesson and User table
LessonUser = KissUserList.merge(Lesson, how = 'left', left_on = 'Identity', right_on = 'identity')
Notmatching = LessonUser[LessonUser['identity'].isnull()]
matching = LessonUser[~LessonUser['identity'].isnull()]

#check if userid can match anything
Notmatching.ix[:,:10].merge(Lesson, how = 'inner', left_on = 'Userid',right_on = 'identity')

,ID,Identity,Userid,Email,IAP Trial Success,First Age,Number Childen,Number Children,Gender,LastAge,eventname,identity,timestamp,manuscriptid,manuscripttitle,subscriber status,unitcategory


In [322]:
matching.head()

,ID,Identity,Userid,Email,IAP Trial Success,First Age,Number Childen,Number Children,Gender,LastAge,...,PAID_6_timestamp,PAID_7_timestamp,TRIAL_timestamp,eventname,identity,timestamp,manuscriptid,manuscripttitle,subscriber status,unitcategory
32,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,None,None,2017-03-12 05:23:13,complete,07008405@hope.ac.uk,2017-03-12 05:44:20,698.0,River Dance,TRIAL/ITUNES,GAM
33,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,None,None,2017-03-12 05:23:13,complete,07008405@hope.ac.uk,2017-03-12 05:41:14,696.0,Pick a Color,TRIAL/ITUNES,GAM
34,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,None,None,2017-03-12 05:23:13,open,07008405@hope.ac.uk,2017-03-12 05:41:22,693.0,Animals in Space!,TRIAL/ITUNES,GAM
35,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,None,None,2017-03-12 05:23:13,open,07008405@hope.ac.uk,2017-03-12 05:43:03,698.0,River Dance,TRIAL/ITUNES,GAM
36,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,None,None,2017-03-12 05:23:13,complete,07008405@hope.ac.uk,2017-03-12 05:38:13,692.0,Max Bananas,TRIAL/ITUNES,GAM


In [323]:
matching['PAID_1_timestamp'] = pd.to_datetime(matching['PAID_1_timestamp'])
matching['PAID_2_timestamp'] = pd.to_datetime(matching['PAID_2_timestamp'])
matching['PAID_3_timestamp'] = pd.to_datetime(matching['PAID_3_timestamp'])
matching['PAID_4_timestamp'] = pd.to_datetime(matching['PAID_4_timestamp'])
matching['PAID_5_timestamp'] = pd.to_datetime(matching['PAID_5_timestamp'])
matching['PAID_6_timestamp'] = pd.to_datetime(matching['PAID_6_timestamp'])
matching['PAID_7_timestamp'] = pd.to_datetime(matching['PAID_7_timestamp'])

In [324]:
matching['days_from_trial'] = [x.days for x in (matching['timestamp'] - matching['IAP Trial Success'])]
matching.loc[~(matching['PAID_1_timestamp'].isnull()),'days_from_PAID_1'] = [x.days for x in (matching.loc[~(matching['PAID_1_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_1_timestamp'].isnull()),'PAID_1_timestamp'])]
matching.loc[~(matching['PAID_2_timestamp'].isnull()),'days_from_PAID_2'] = [x.days for x in (matching.loc[~(matching['PAID_2_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_2_timestamp'].isnull()),'PAID_2_timestamp'])]
matching.loc[~(matching['PAID_3_timestamp'].isnull()),'days_from_PAID_3'] = [x.days for x in (matching.loc[~(matching['PAID_3_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_3_timestamp'].isnull()),'PAID_3_timestamp'])]
matching.loc[~(matching['PAID_4_timestamp'].isnull()),'days_from_PAID_4'] = [x.days for x in (matching.loc[~(matching['PAID_4_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_4_timestamp'].isnull()),'PAID_4_timestamp'])]
matching.loc[~(matching['PAID_5_timestamp'].isnull()),'days_from_PAID_5'] = [x.days for x in (matching.loc[~(matching['PAID_5_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_5_timestamp'].isnull()),'PAID_5_timestamp'])]
matching.loc[~(matching['PAID_6_timestamp'].isnull()),'days_from_PAID_6'] = [x.days for x in (matching.loc[~(matching['PAID_6_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_6_timestamp'].isnull()),'PAID_6_timestamp'])]
matching.loc[~(matching['PAID_7_timestamp'].isnull()),'days_from_PAID_7'] = [x.days for x in (matching.loc[~(matching['PAID_7_timestamp'].isnull()),'timestamp'] - matching.loc[~(matching['PAID_7_timestamp'].isnull()),'PAID_7_timestamp'])]


In [325]:
matching.head(3)

,ID,Identity,Userid,Email,IAP Trial Success,First Age,Number Childen,Number Children,Gender,LastAge,...,subscriber status,unitcategory,days_from_trial,days_from_PAID_1,days_from_PAID_2,days_from_PAID_3,days_from_PAID_4,days_from_PAID_5,days_from_PAID_6,days_from_PAID_7
32,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [326]:
#remove testing account
matching = matching[['homerapp' not in str(Email) for Email in matching['Email']]]
matching = matching[['homerlearning' not in str(Email) for Email in matching['Email']]]
matching = matching[['speakaboos' not in str(Email) for Email in matching['Email']]]
matching = matching[['homer' not in str(Email) for Email in matching['Email']]]

In [338]:
#Remove number of children > 1
matching_onechild = matching[matching['Number Childen']==1]
print(len(matching_onechild))

2009510


In [339]:
matching_onechild.head()

,ID,Identity,Userid,Email,IAP Trial Success,First Age,Number Childen,Number Children,Gender,LastAge,...,subscriber status,unitcategory,days_from_trial,days_from_PAID_1,days_from_PAID_2,days_from_PAID_3,days_from_PAID_4,days_from_PAID_5,days_from_PAID_6,days_from_PAID_7
32,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,07008405@hope.ac.uk,2017-03-12 05:23:16,4,1.0,NaN,NaN,4,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
matching_onechild.drop(['Email','Number Children','Gender','LastAge'], axis = 1, inplace = True)

## Deal with app version data

In [341]:
appversion_merge = KissUserList[['Identity']].merge(appversion_data, how = 'left', left_on = 'Identity', right_on = 'person_id')

In [342]:
appversion_merge = appversion_merge.drop_duplicates()

In [343]:
appversion_final = appversion_merge.groupby(['Identity'])['appversion_property'].value_counts().unstack(fill_value = 0)

In [344]:
appversion_final.head()

appversion_property,3.910,3.930,3.970,3.980,4.0.0,4.0.1,4.0.2,4.0.3
Identity,,,,,,,,
codyfancy@yahoo.com,1,0,1,1,1,1,1,1
theresaannlaham@gmail.com,0,0,0,0,0,0,1,0
0BUczzvSvr,0,0,0,0,1,0,0,0
104360143@qq.com,0,0,1,1,1,1,0,0
104522025@qq.com,0,0,0,1,0,0,0,0


In [335]:
#matching_onechild = matching_onechild.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

In [345]:
matching_onechild.head()

,ID,Identity,Userid,IAP Trial Success,First Age,Number Childen,Pathways manuscript open,Welcome_page,PAID_1_timestamp,PAID_2_timestamp,...,subscriber status,unitcategory,days_from_trial,days_from_PAID_1,days_from_PAID_2,days_from_PAID_3,days_from_PAID_4,days_from_PAID_5,days_from_PAID_6,days_from_PAID_7
32,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,2017-03-12 05:23:16,4,1.0,NaN,NaN,NaT,NaT,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,2017-03-12 05:23:16,4,1.0,NaN,NaN,NaT,NaT,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,2017-03-12 05:23:16,4,1.0,NaN,NaN,NaT,NaT,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,2017-03-12 05:23:16,4,1.0,NaN,NaN,NaT,NaT,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,3221753,07008405@hope.ac.uk,EGIzcpRPZ6,2017-03-12 05:23:16,4,1.0,NaN,NaN,NaT,NaT,...,TRIAL/ITUNES,GAM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Code a function to automate data preparation

In [1202]:
#This is for Day0, week 1, which is not 'split-up' data
def data_preparation(data, KissUserList, start, end, eventname, group_by):
    record = data[(data['days_from_trial']>=start) & (data['days_from_trial']<=end) & (data['eventname'] ==eventname)]  
    usage = record.groupby(['identity'])[group_by].value_counts().unstack(fill_value = 0)
    usage['If_return'] = 0
    return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
    usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    if group_by == 'unitcategory':
        record_drop_duplicate = record.drop_duplicates(subset = ['identity','unitcategory','manuscripttitle'])
        usage_unique = record_drop_duplicate.groupby(["identity"])['unitcategory'].value_counts().unstack(fill_value = 0)
        usage_unique.rename(columns = lambda x: x + '_unique', inplace = True)
        usage = usage.merge(usage_unique, how = 'inner', left_index = True, right_index = True)
    usage.reset_index(inplace = True)
    usage = usage.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
    usage.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
    return usage

In [1203]:
df = data_preparation(matching_onechild, KissUserList, start = 0, end = 30, eventname = 'complete',group_by = 'unitcategory')

In [1183]:
#This is for CTP, M1,M2, ....M6, which requires split-up data
def data_preparation1(data, KissUserList, start, end, eventname, group_by):
    record = data[(data['days_from_trial']>=start) & (data['days_from_trial']<=end) & (data['eventname'] ==eventname)]  
    usage = record.groupby(['identity'])[group_by].value_counts().unstack(fill_value = 0)
    usage['If_return'] = 0
    return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
    usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    if group_by == 'unitcategory':
        record_drop_duplicate = record.drop_duplicates(subset = ['identity','unitcategory','manuscripttitle'])
        usage_unique = record_drop_duplicate.groupby(["identity"])['unitcategory'].value_counts().unstack(fill_value = 0)
        usage_unique.rename(columns = lambda x: x + '_unique', inplace = True)
        usage = usage.merge(usage_unique, how = 'inner', left_index = True, right_index = True)
    #usage.reset_index(inplace = True)
    #usage = usage.merge(KissUserList, how = 'left', left_index = True, right_on = 'Identity')
    #usage.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
    return usage

In [346]:
#This is for CTP, M1,M2, ....M6, which requires split-up data
def data_preparation2(data, KissUserList, start, end, eventname, group_by, time_from, return_indicator):
    record = data[(data[time_from]>=start) & (data[time_from]<=end) & (data['eventname'] ==eventname)]  
    usage = record.groupby(['Identity'])[group_by].value_counts().unstack(fill_value = 0)
    #usage['If_return'] = 0
    #return_user = set(data[~data[return_indicator].isnull()]['identity'])
    #usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    if group_by == 'unitcategory':
        record_drop_duplicate = record.drop_duplicates(subset = ['Identity','unitcategory','manuscripttitle'])
        usage_unique = record_drop_duplicate.groupby(["Identity"])['unitcategory'].value_counts().unstack(fill_value = 0)
        usage_unique.rename(columns = lambda x: x + '_unique', inplace = True)
        usage = usage.merge(usage_unique, how = 'inner', left_index = True, right_index = True)
    #usage.reset_index(inplace = True)
    #usage = usage.merge(KissUserList, how = 'left', left_index = True, right_on = 'Identity')
    #usage.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
    return usage

In [347]:
data_preparation2(matching_onechild, KissUserList, 0, 30, 'open', 'unitcategory', 'days_from_trial','PAID_1_timestamp').info()

<class 'pandas.core.frame.DataFrame'>
Index: 12906 entries, 07008405@hope.ac.uk to zyxyhm@163.com
Data columns (total 32 columns):
DECOD                                  12906 non-null int64
DTW                                    12906 non-null int64
FR                                     12906 non-null int64
GAM                                    12906 non-null int64
LTR                                    12906 non-null int64
MUS                                    12906 non-null int64
OUT_OF_PICKLES                         12906 non-null int64
Pathway-DTW                            12906 non-null int64
Pathway-FRE                            12906 non-null int64
Pathway-GAM                            12906 non-null int64
RAS                                    12906 non-null int64
SON                                    12906 non-null int64
SaturdaySurpriseBanner                 12906 non-null int64
SaturdaySurpriseInterstitial           12906 non-null int64
StoryTimeFeaturedImage       

In [401]:
#This is for CTP, M1,M2, ....M6, which requires split-up data
def data_preparation_interval(data, KissUserList,start, end, interval, eventname, group_by, time_from, return_indicator):
    df = {}
    start_stamp = start
    end_stamp = 0
    interate = math.floor((end-start+1)/interval)
    modula = (end-start+1)%interval
    for i in range(interate +1):
        if i == interate:
            end_stamp = end
            start_stamp = start
            result = data_preparation2(data, KissUserList, start_stamp, end_stamp, eventname = eventname, group_by = group_by,time_from = time_from,return_indicator = return_indicator).rename(columns = lambda x:x + '_total')
        elif i == interate - 1:
            end_stamp = end
            result = data_preparation2(data, KissUserList, start_stamp, end_stamp, eventname = eventname, group_by = group_by,time_from = time_from,return_indicator = return_indicator).rename(columns = lambda x:x + '_week_' + str(i))
        else: #rename(columns = lambda x: x + 'week_' + str(i)
            end_stamp = start_stamp + interval-1
            result = data_preparation2(data, KissUserList, start_stamp, end_stamp, eventname = eventname, group_by = group_by,time_from = time_from,return_indicator = return_indicator).rename(columns = lambda x:x + '_week_' + str(i))
        start_stamp = start_stamp + interval
        df.update({'week'+str(i):result})     
    return df
                                                                                                                                                                                                                       

In [348]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 30, interval = 6, eventname = 'complete',group_by = 'unitcategory',time_from = 'days_from_trial', return_indicator = 'PAID_1_timestamp')

In [352]:
#Combine all the data into one dataframe
m0 = df['week0'].merge(df['week1'],
                 how = 'outer',
                 left_index = True,
                 right_index = True).merge(df['week2'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week3'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week4'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True)

m0['If_return'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_1_timestamp'].isnull()]['Identity'])
m0.loc[m0.index.isin(return_user),'If_return'] = 1 #53xx return on Day

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)


In [354]:
m0.head()

unitcategory,DECOD_week_0,DTW_week_0,GAM_week_0,LTR_week_0,MUS_week_0,OUT_OF_PICKLES_week_0,Pathway-DTW_week_0,Pathway-FRE_week_0,Pathway-GAM_week_0,RAS_week_0,...,Pathway-DTW_unique_m0,Pathway-FRE_unique_m0,Pathway-GAM_unique_m0,RAS_unique_m0,SON_unique_m0,SaturdaySurpriseBanner_unique_m0,SaturdaySurpriseInterstitial_unique_m0,StoryTimeFeaturedImage_unique_m0,onboarding_unique_m0,If_return
Identity,,,,,,,,,,,,,,,,,,,,,
07008405@hope.ac.uk,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
0xigensun@gmail.com,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1019044059@qq.com,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,2,0,0,0,0,1
1020newyork@gmail.com,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0,0,0,12,1,0,0,0,0,1
10299422@qq.com,0.0,4.0,5.0,11.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,0,14,10,0,0,0,0,0


In [355]:
m0.reset_index(inplace = True)

In [356]:
m0 = m0.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')

In [357]:
m0 = m0.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

In [360]:
m0.columns[120:]

Index(['Pathway-GAM_unique_m0', 'RAS_unique_m0', 'SON_unique_m0',
       'SaturdaySurpriseBanner_unique_m0',
       'SaturdaySurpriseInterstitial_unique_m0',
       'StoryTimeFeaturedImage_unique_m0', 'onboarding_unique_m0', 'If_return',
       'ID', 'Userid', 'Email', 'IAP Trial Success', 'First Age',
       'Number Childen', 'Number Children', 'Gender', 'LastAge',
       'Pathways manuscript open', 'Welcome_page', 'PAID_1_timestamp',
       'PAID_2_timestamp', 'PAID_3_timestamp', 'PAID_4_timestamp',
       'PAID_5_timestamp', 'PAID_6_timestamp', 'PAID_7_timestamp',
       'TRIAL_timestamp', '3.910', '3.930', '3.970', '3.980', '4.0.0', '4.0.1',
       '4.0.2', '4.0.3'],
      dtype='object')

In [362]:
m0.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge'], axis = 1, inplace = True)

In [363]:
m0

,Identity,DECOD_week_0,DTW_week_0,GAM_week_0,LTR_week_0,MUS_week_0,OUT_OF_PICKLES_week_0,Pathway-DTW_week_0,Pathway-FRE_week_0,Pathway-GAM_week_0,...,PAID_7_timestamp,TRIAL_timestamp,3.910,3.930,3.970,3.980,4.0.0,4.0.1,4.0.2,4.0.3
0,07008405@hope.ac.uk,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-03-12 05:23:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xigensun@gmail.com,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-02-06 17:16:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1019044059@qq.com,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-02-07 22:23:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1020newyork@gmail.com,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-02-14 18:31:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10299422@qq.com,0.0,4.0,5.0,11.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-01-07 05:57:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,104360143@qq.com,0.0,17.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-01-25 00:23:45,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
6,10445987@qq.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2017-02-07 07:59:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,104522025@qq.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2017-02-27 00:58:06,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,107849680@qq.com,0.0,5.0,4.0,18.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-02-06 02:50:06,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
9,10elawncare@att.net,0.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,...,None,2017-03-04 07:50:17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Day1 X Category

In [1229]:
day0_category_usage = data_preparation(matching_onechild, KissUserList, 0, 0, 'open', 'unitcategory')

In [756]:
pd.DataFrame.to_csv(day0_category_usage,'Process_Data/day0_category_usage_complete.csv')

## Day1 X title

In [786]:
day0_title_usage = data_preparation(matching_onechild, KissUserList, 0, 0, 'open', 'manuscripttitle')

,identity,A Bat and a Bat,A Big Bed,A Big Duck,A Big Jam,A Day in Ancient Egypt,A Dewdrop,A Farm-Picture,A Good Bell,A Good Plan,...,"Yes, You Can",Yip and Tutt Look,Yip the Yak,You're Happy And You Know It: Drawing,You're a Grand Old Flag,"Zip, Zip, Zip",Zookeepers,Day1_return,First Age,Gender
0,07008405@hope.ac.uk,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,NaN
1,0xigensun@gmail.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,M
2,1019044059@qq.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,F
3,1020newyork@gmail.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,5,M
4,102724871@qq.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,NaN


In [787]:
pd.DataFrame.to_csv(day0_title_usage,'Process_Data/day0_title_usage_open.csv')

## Week 1 X Category

In [1231]:
week0_category_usage = data_preparation(matching_onechild, KissUserList, 0, 6, 'open', 'unitcategory')

In [937]:
pd.DataFrame.to_csv(week0_category_usage,'Process_Data/week0_category_usage_complete.csv')

## Week 1 X Title

In [1232]:
week0_title_usage = data_preparation(matching_onechild, KissUserList, 0, 6, 'open', 'manuscripttitle')

In [1233]:
week0_title_usage.head()

,identity,A Bat and a Bat,A Big Bed,A Big Duck,A Big Jam,A Day in Ancient Egypt,A Dewdrop,A Farm-Picture,A Good Bell,A Good Plan,...,"Yes, You Can",Yip and Tutt Look,Yip the Yak,You're Happy And You Know It: Drawing,You're a Grand Old Flag,"Zip, Zip, Zip",Zookeepers,If_return,First Age,Gender
0,07008405@hope.ac.uk,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,4,NaN
1,0xigensun@gmail.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,M
2,1019044059@qq.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,F
3,1020newyork@gmail.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,M
4,102724871@qq.com,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,NaN


In [831]:
pd.DataFrame.to_csv(week0_title_usage,'Process_Data/week0_title_usage_complete.csv')

## CTP (M1)  X Category

In [1239]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 30, interval = 6, eventname = 'complete',group_by = 'unitcategory')

#Combine all the data into one dataframe
m0_category_usage = df['week0'].merge(df['week1'],
                 how = 'outer',
                 left_index = True,
                 right_index = True).merge(df['week2'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week3'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week4'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True)
m0_category_usage['If_return'] = 0
return_user = set(matching_onechild[(matching_onechild['days_from_trial']>30) & (matching_onechild['days_from_trial']<=60)]['identity'])
m0_category_usage.loc[m0_category_usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day

m0_category_usage.reset_index(inplace = True)
m0_category_usage = m0_category_usage.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
m0_category_usage.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)

In [1241]:
m0_category_usage.head()

,identity,DECOD_week_0,DTW_week_0,GAM_week_0,LTR_week_0,MUS_week_0,OUT_OF_PICKLES_week_0,Pathway-DTW_week_0,Pathway-FRE_week_0,Pathway-GAM_week_0,...,Pathway-GAM_unique_m0,RAS_unique_m0,SON_unique_m0,SaturdaySurpriseBanner_unique_m0,SaturdaySurpriseInterstitial_unique_m0,StoryTimeFeaturedImage_unique_m0,onboarding_unique_m0,If_return,First Age,Gender
0,07008405@hope.ac.uk,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,4,NaN
1,0xigensun@gmail.com,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,3,M
2,1019044059@qq.com,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0.0,...,0,0,2,0,0,0,0,0,5,F
3,1020newyork@gmail.com,0.0,0.0,0.0,389.0,0.0,0.0,0.0,0.0,0.0,...,0,12,1,0,0,0,0,0,5,M
4,10299422@qq.com,0.0,4.0,5.0,131.0,0.0,0.0,0.0,0.0,0.0,...,0,14,10,0,0,0,0,1,4,NaN


In [891]:
pd.DataFrame.to_csv(m0_category_usage,'Process_Data/m0_category_usage_complete.csv')

## CTP (M1)  X Title

In [1245]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 30, interval = 6, eventname = 'complete',group_by = 'manuscripttitle')

#Combine all the data into one dataframe
m0_title_usage = df['week0'].merge(df['week1'],
                 how = 'outer',
                 left_index = True,
                 right_index = True).merge(df['week2'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week3'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week4'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True)
m0_title_usage['If_return'] = 0
return_user = set(matching_onechild[(matching_onechild['days_from_trial']>30) & (matching_onechild['days_from_trial']<=60)]['identity'])
m0_title_usage.loc[m0_category_usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day

m0_title_usage.reset_index(inplace = True)
m0_title_usage = m0_title_usage.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
m0_title_usage.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)

In [1247]:
m0_title_usage.head()

,identity,A Bat and a Bat_week_0,A Big Bed_week_0,A Big Duck_week_0,A Big Jam_week_0,A Day in Ancient Egypt_week_0,A Dewdrop_week_0,A Farm-Picture_week_0,A Good Bell_week_0,A Good Plan_week_0,...,Yip and Tutt Look_m0,Yip the Yak_m0,You're Happy And You Know It: Drawing_m0,You're a Grand Old Flag_m0,"Zip, Zip, Zip_m0",Zookeepers_m0,If_return_m0,If_return,First Age,Gender
0,07008405@hope.ac.uk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,4,NaN
1,0xigensun@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,3,M
2,1019044059@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,5,F
3,1020newyork@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,5,M
4,10299422@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,0,4,NaN


In [911]:
pd.DataFrame.to_csv(m0_title_usage,'Process_Data/m0_title_usage_complete.csv')

# Subscription Retention

## CTP Retention

## CTP X Category

In [391]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 30, interval = 7, eventname = 'complete',group_by = 'unitcategory',time_from = 'days_from_trial', return_indicator = 'PAID_1_timestamp')

In [392]:
df.keys()

dict_keys(['week0', 'week1', 'week2', 'week3', 'week4'])

In [393]:
#Combine all the data into one dataframe
m0 = df['week0'].merge(df['week1'],
                 how = 'outer',
                 left_index = True,
                 right_index = True).merge(df['week2'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week3'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week4'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True)

m0['If_return'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_1_timestamp'].isnull()]['Identity'])
m0.loc[m0.index.isin(return_user),'If_return'] = 1 #53xx return on Day

m0.reset_index(inplace = True)
m0 = m0.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
m0 = m0.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
m0.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

In [394]:
m0.columns[120:]

Index(['Pathway-GAM_unique_m0', 'RAS_unique_m0', 'SON_unique_m0',
       'SaturdaySurpriseBanner_unique_m0',
       'SaturdaySurpriseInterstitial_unique_m0',
       'StoryTimeFeaturedImage_unique_m0', 'onboarding_unique_m0', 'If_return',
       'First Age', 'Pathways manuscript open', 'Welcome_page', '3.910',
       '3.930', '3.970', '3.980', '4.0.0', '4.0.1', '4.0.2', '4.0.3'],
      dtype='object')

In [376]:
len(m0)

11503

## CTP X Manuscript

In [379]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 30, interval = 7, eventname = 'complete',group_by = 'manuscripttitle',time_from = 'days_from_trial', return_indicator = 'PAID_1_timestamp')

In [380]:
#Combine all the data into one dataframe
CTP_title_complete = df['week0'].merge(df['week1'],
                 how = 'outer',
                 left_index = True,
                 right_index = True).merge(df['week2'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week3'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True).merge(df['week4'],
                                           how = 'outer',
                                           left_index = True,
                                           right_index = True)

CTP_title_complete['If_return'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_1_timestamp'].isnull()]['Identity'])
CTP_title_complete.loc[CTP_title_complete.index.isin(return_user),'If_return'] = 1 #53xx return on Day

CTP_title_complete.reset_index(inplace = True)
CTP_title_complete = CTP_title_complete.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
CTP_title_complete = CTP_title_complete.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
CTP_title_complete.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

In [381]:
CTP_title_complete

,Identity,A Bat and a Bat_week_0,A Big Bed_week_0,A Big Duck_week_0,A Big Jam_week_0,A Day in Ancient Egypt_week_0,A Dewdrop_week_0,A Farm-Picture_week_0,A Good Bell_week_0,A Good Plan_week_0,...,Pathways manuscript open,Welcome_page,3.910,3.930,3.970,3.980,4.0.0,4.0.1,4.0.2,4.0.3
0,07008405@hope.ac.uk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xigensun@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1019044059@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1020newyork@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10299422@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,104360143@qq.com,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
6,10445987@qq.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,104522025@qq.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,107849680@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"Oct 01, 2017 21:14:51",NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
9,10elawncare@att.net,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# M1 cancellation X Category

In [402]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 60, interval = 7, eventname = 'complete',group_by = 'unitcategory',time_from = 'days_from_trial', return_indicator = 'PAID_2_timestamp')

In [403]:
df.keys()

dict_keys(['week0', 'week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8'])

In [404]:
#Combine all the data into one dataframe
df_combine = df['week0']
for i in ['week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8']:
    df_combine = df_combine.merge(df[i],
                     how = 'outer',
                     left_index = True,
                     right_index = True)

In [407]:
df_combine.head()

unitcategory,DECOD_week_0,DTW_week_0,GAM_week_0,LTR_week_0,MUS_week_0,OUT_OF_PICKLES_week_0,Pathway-DTW_week_0,Pathway-FRE_week_0,Pathway-GAM_week_0,RAS_week_0,...,OUT_OF_PICKLES_unique_total,Pathway-DTW_unique_total,Pathway-FRE_unique_total,Pathway-GAM_unique_total,RAS_unique_total,SON_unique_total,SaturdaySurpriseBanner_unique_total,SaturdaySurpriseInterstitial_unique_total,StoryTimeFeaturedImage_unique_total,onboarding_unique_total
Identity,,,,,,,,,,,,,,,,,,,,,
07008405@hope.ac.uk,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
0xigensun@gmail.com,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1019044059@qq.com,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,2,0,0,0,0
1020newyork@gmail.com,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0,0,0,0,12,1,0,0,0,0
10299422@qq.com,0.0,4.0,5.0,11.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,0,0,14,11,0,0,0,0


In [414]:
#Combine all the data into one dataframe

df_combine['If_return'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_2_timestamp'].isnull()]['Identity'])
df_combine.loc[df_combine.index.isin(return_user),'If_return'] = 1 #53xx return on Day

df_combine.reset_index(inplace = True)
df_combine = df_combine.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
m1_category = df_combine.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
m1_category.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

In [410]:
m1_manuscript.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11692 entries, 0 to 11691
Columns: 239 entries, Identity to 4.0.3
dtypes: float64(202), int64(33), object(4)
memory usage: 21.4+ MB


## M1 X manuscript 

In [416]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 60, interval = 7, eventname = 'complete',group_by = 'manuscripttitle',time_from = 'days_from_trial', return_indicator = 'PAID_2_timestamp')


In [412]:
df.keys()

dict_keys(['week0', 'week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8'])

In [417]:
#Combine all the data into one dataframe
df_combine = df['week0']
for i in ['week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8']:
    df_combine = df_combine.merge(df[i],
                     how = 'outer',
                     left_index = True,
                     right_index = True)

In [421]:
#Combine all the data into one dataframe

df_combine['If_return'] = 0
df_combine['Paid_1'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_2_timestamp'].isnull()]['Identity'])
m1_user = set(matching_onechild[~matching_onechild['PAID_1_timestamp'].isnull()]['Identity'])
df_combine.loc[df_combine.index.isin(return_user),'If_return'] = 1 #53xx return on Day
df_combine.loc[df_combine.index.isin(m1_user),'Paid_1'] = 1 #53xx return on Day

df_combine.reset_index(inplace = True)
df_combine = df_combine.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
m1_manuscript = df_combine.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
m1_manuscript.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

ValueError: labels ['ID' 'Userid' 'Email' 'IAP Trial Success' 'Number Childen'
 'Number Children' 'Gender' 'LastAge' 'PAID_1_timestamp' 'PAID_2_timestamp'
 'PAID_3_timestamp' 'PAID_4_timestamp' 'PAID_5_timestamp'
 'PAID_6_timestamp' 'PAID_7_timestamp' 'TRIAL_timestamp'] not contained in axis

In [419]:
m1_manuscript.head()

,Identity,A Bat and a Bat_week_0,A Big Bed_week_0,A Big Duck_week_0,A Big Jam_week_0,A Day in Ancient Egypt_week_0,A Dewdrop_week_0,A Farm-Picture_week_0,A Good Bell_week_0,A Good Plan_week_0,...,Pathways manuscript open,Welcome_page,3.910,3.930,3.970,3.980,4.0.0,4.0.1,4.0.2,4.0.3
0,07008405@hope.ac.uk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xigensun@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1019044059@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1020newyork@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10299422@qq.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## M2 Cancellation X Category

In [422]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 90, interval = 30, eventname = 'complete',group_by = 'unitcategory',time_from = 'days_from_trial', return_indicator = 'PAID_3_timestamp')


In [425]:
#Combine all the data into one dataframe
df_combine = df['week0']
for i in ['week1', 'week2', 'week3']:
    df_combine = df_combine.merge(df[i],
                     how = 'outer',
                     left_index = True,
                     right_index = True)

In [426]:
#Combine all the data into one dataframe

df_combine['If_return'] = 0
df_combine['Paid_2'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_3_timestamp'].isnull()]['Identity'])
m1_user = set(matching_onechild[~matching_onechild['PAID_2_timestamp'].isnull()]['Identity'])
df_combine.loc[df_combine.index.isin(return_user),'If_return'] = 1 #53xx return on Day
df_combine.loc[df_combine.index.isin(m1_user),'Paid_2'] = 1 #53xx return on Day

df_combine.reset_index(inplace = True)
df_combine = df_combine.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
m1_category = df_combine.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
m1_category.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

In [435]:
m1_manuscript['If_return'].value_counts()

0    8197
1    3576
Name: If_return, dtype: int64

## M2 Cancellation X manuscript

In [436]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 90, interval = 30, eventname = 'complete',group_by = 'manuscripttitle',time_from = 'days_from_trial', return_indicator = 'PAID_3_timestamp')


In [437]:
df.keys()

dict_keys(['week0', 'week1', 'week2', 'week3'])

In [438]:
#Combine all the data into one dataframe
df_combine = df['week0']
for i in ['week1', 'week2', 'week3']:
    df_combine = df_combine.merge(df[i],
                     how = 'outer',
                     left_index = True,
                     right_index = True)

In [439]:
#Combine all the data into one dataframe

df_combine['If_return'] = 0
df_combine['Paid_2'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_3_timestamp'].isnull()]['Identity'])
m1_user = set(matching_onechild[~matching_onechild['PAID_2_timestamp'].isnull()]['Identity'])
df_combine.loc[df_combine.index.isin(return_user),'If_return'] = 1 #53xx return on Day
df_combine.loc[df_combine.index.isin(m1_user),'Paid_2'] = 1 #53xx return on Day

df_combine.reset_index(inplace = True)
df_combine = df_combine.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
m1_category = df_combine.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
m1_category.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

In [440]:
m1_manuscript['If_return'].value_counts()

0    8197
1    3576
Name: If_return, dtype: int64

## M3 Cancellation X Category

In [441]:
df = data_preparation_interval(matching_onechild, KissUserList, start = 0, end = 120, interval = 30, eventname = 'complete',group_by = 'unitcategory',time_from = 'days_from_trial', return_indicator = 'PAID_4_timestamp')


In [442]:
df.keys()

dict_keys(['week0', 'week1', 'week2', 'week3', 'week4'])

In [443]:
#Combine all the data into one dataframe
df_combine = df['week0']
for i in ['week1', 'week2', 'week3','week4']:
    df_combine = df_combine.merge(df[i],
                     how = 'outer',
                     left_index = True,
                     right_index = True)

In [444]:
#Combine all the data into one dataframe

df_combine['If_return'] = 0
df_combine['Paid_3'] = 0
return_user = set(matching_onechild[~matching_onechild['PAID_4_timestamp'].isnull()]['Identity'])
m1_user = set(matching_onechild[~matching_onechild['PAID_3_timestamp'].isnull()]['Identity'])
df_combine.loc[df_combine.index.isin(return_user),'If_return'] = 1 #53xx return on Day
df_combine.loc[df_combine.index.isin(m1_user),'Paid_3'] = 1 #53xx return on Day

df_combine.reset_index(inplace = True)
df_combine = df_combine.merge(KissUserList, how = 'left', left_on = 'Identity', right_on = 'Identity')
m1_category = df_combine.merge(appversion_final, how = 'left', left_on = 'Identity', right_index = True)

#usage['If_return'] = 0
#return_user = set(data[(data['days_from_trial']>end) & (data['days_from_trial']<=end + (end-start))]['identity'])
#usage.loc[usage.index.isin(return_user),'If_return'] = 1 #53xx return on Day
    
#m0.reset_index(inplace = True)
#m0 = m0.merge(KissUserList, how = 'left', left_on = 'identity', right_on = 'Identity')
#m0.drop(['ID','Identity','Userid','Email','IAP Trial Success','Number Childen','Number Children','LastAge'],axis = 1, inplace = True)
m1_category.drop(['ID','Userid','Email','IAP Trial Success','Number Childen', 'Number Children','Gender', 'LastAge',
        'PAID_1_timestamp','PAID_2_timestamp','PAID_3_timestamp','PAID_4_timestamp','PAID_5_timestamp',
        'PAID_6_timestamp','PAID_7_timestamp','TRIAL_timestamp'], axis = 1, inplace = True)

In [446]:
m1_category['Paid_3'].value_counts()

0    8214
1    3602
Name: Paid_3, dtype: int64

In [447]:
m1_category['If_return'].value_counts()

0    8812
1    3004
Name: If_return, dtype: int64